# 1. Image resizing and normalization

In [4]:
## The dataset of dogs & cats images was downloaded from Kaggle https://www.kaggle.com/datasets/chetankv/dogs-cats-images

## Using OpenCV we: 
## a) Resize all the images and 
## b) Normalize the pixel values of the images. Normalization is a process that scales the pixel values 
## of an image to a specific range. It helps to improve the performance of the ML model 
## by reducing the scale of the input data.

import os
import cv2

# Define the new size
new_size = (256, 256)

# Define the path to the dataset folder
dataset_path = "dataset/"

# Loop through the training and test sets
for set_name in ["training_set", "test_set"]:
    for class_name in ["cats", "dogs"]:
        # Define the path to the class folder
        path = dataset_path + set_name + "/" + class_name + "/"
        # Loop through all the images in the class folder
        for filename in os.listdir(path):
            # Load the image
            img = cv2.imread(path + filename)
            # Resize the image
            img = cv2.resize(img, new_size)
            # Normalize the pixel values
            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
            # Save the resized image
            cv2.imwrite(path + filename, img)


## 2. Data augmentation

Data augmentation is a technique that is used to artificially increase the size of the training dataset by creating new, modified versions of the existing images. This helps to improve the performance of the machine learning model by increasing the diversity of the training data and reducing overfitting.

In [1]:
 ## This code uses the Keras ImageDataGenerator class to perform data augmentation on the images, 
 ## and for each existing image it generates 5 new images using different combinations of data augmentation 
 ## options such as rotation, shift, zoom, and flipping.

from keras.preprocessing.image import ImageDataGenerator
import scipy
import os
import cv2

# Define the path to the dataset folder
dataset_path = "dataset/"

# Define the data generator
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Loop through the training and test sets
for set_name in ["training_set", "test_set"]:
    for class_name in ["cats", "dogs"]:
        # Define the path to the class folder
        path = dataset_path + set_name + "/" + class_name + "/"
        # Loop through all the images in the class folder
        for filename in os.listdir(path):
            # Load the image
            img = cv2.imread(path + filename)
            # Reshape the image
            img = img.reshape((1,) + img.shape)
            # Generate new images
            i = 0
            for batch in datagen.flow(img, batch_size=1, save_prefix=class_name + '_' + filename, save_format='jpg', save_to_dir=path):
                i += 1
                if i > 5:
                    break

## 3. Building, Training and Testing a Neural Network for Classification

In [14]:
## We build a simple convolutional neural network (CNN) model for classifying images of cats and dogs

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Initialize the model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output data for the dense layers
model.add(Flatten())

# Add a dense layer for classification
model.add(Dense(64, activation='relu'))
# Add a final sigmoid layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

#tf.function
def train_and_evaluate(model, datagen):
    # Load the preprocessed data
    train_data = datagen.flow_from_directory(directory='dataset/training_set', target_size=(256, 256), batch_size=32, class_mode='binary')
    test_data = datagen.flow_from_directory(directory='dataset/test_set', target_size=(256, 256), batch_size=32, class_mode='binary')

    # Train the model on the new data for a few epochs
    model.fit(train_data, epochs=5, batch_size=64)

    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_data)
    print("Test accuracy:", test_acc)

# call the function with the model and data generators
train_and_evaluate(model, datagen)

Found 55989 images belonging to 2 classes.
Found 13995 images belonging to 2 classes.
Epoch 1/5
1750/1750 [==============================] - 3534s 2s/step - loss: 0.6041 - accuracy: 0.6613
Epoch 2/5
1750/1750 [==============================] - 3944s 2s/step - loss: 0.4865 - accuracy: 0.7636
Epoch 3/5
1750/1750 [==============================] - 4097s 2s/step - loss: 0.3916 - accuracy: 0.8196
Epoch 4/5
1750/1750 [==============================] - 3870s 2s/step - loss: 0.2595 - accuracy: 0.8906
Epoch 5/5
438/438 [==============================] - 317s 723ms/step - loss: 0.7955 - accuracy: 0.7655
Test accuracy: 0.7654876708984375
